In [ ]:
## Importing packages

# This R environment comes with all of CRAN and many other helpful packages preinstalled.
# You can see which packages are installed by checking out the kaggle/rstats docker image: 
# https://github.com/kaggle/docker-rstats

library(tidyverse) # metapackage with lots of helpful functions

## Running code

# In a notebook, you can run a single code cell by clicking in the cell and then hitting 
# the blue arrow to the left, or by clicking in the cell and pressing Shift+Enter. In a script, 
# you can run code by highlighting the code you want to run and then clicking the blue arrow
# at the bottom of this window.

## Reading in files

# You can access files from datasets you've added to this kernel in the "../input/" directory.
# You can see the files added to this kernel by running the code below. 

list.files(path = "../input")

## Saving data

# If you save any files or images, these will be put in the "output" directory. You 
# can see the output directory by committing and running your kernel (using the 
# Commit & Run button) and then checking out the compiled version of your kernel.

In [ ]:
install.packages("factoextra")

In [ ]:
library(factoextra)

In [ ]:
Data <- read_csv("../input/wine_new.csv")

In [ ]:
install.packages('caTools') 
library(caTools) 
  
set.seed(123) 
split = sample.split(Data, SplitRatio = 0.75) 
  
training_set = subset(Data, split == TRUE) 
test_set = subset(Data, split == FALSE) 

In [ ]:
#W.pca <- princomp(training_set[,-1], cor = TRUE, scores = TRUE, covmat = NULL)
W.pca <- prcomp(training_set[,-1],scale.=T)

summary(W.pca)


In [ ]:
W.pca$loadings

The above table shows how individual components contribute to covariance matrix.

In [ ]:
plot(W.pca)

In [ ]:
#Scree Plot
fviz_eig(W.pca)

In [ ]:
'''std_dev=W.pca$sde
pr_var <- std_dev^2
prop_varex <- pr_var/sum(pr_var)
#scree plot
plot(prop_varex, xlab = "Principal Component",
             ylab = "Proportion of Variance Explained",
             type = "b")'''

#Above Plot sufficient

From the analysis of scree plot, we observe that the number of components containing maximum information are 8.

In [ ]:
#cumulative scree plot
plot(cumsum(prop_varex), xlab = "Principal Component",
              ylab = "Cumulative Proportion of Variance Explained",
              type = "b")

The cumulative scree plot shows that the first 7 compponents contain about 90% of information and hence 13 components can be reduced to 7 components.


In [ ]:
fviz_pca_ind(W.pca,
             col.ind = "cos2", # Color by the quality of representation
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE     # Avoid text overlapping
             )

The above plot shows graph of individuals. Individuals with a similar profile are grouped together.

In [ ]:
fviz_pca_var(W.pca,
             col.var = "contrib", # Color by contributions to the PC
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE     # Avoid text overlapping
             )

The above plot shows graph of variables. Positive correlated variables point to the same side of the plot. Negative correlated variables point to opposite sides of the graph.

In [ ]:
fviz_pca_biplot(W.pca, repel = TRUE,
                col.var = "#2E9FDF", # Variables color
                col.ind = "#696969"  # Individuals color
                )

Biplot of individuals and variables

In [ ]:
train.data <- data.frame(Wine=training_set$Wine, W.pca$x)
train.data <- train.data[,1:7]


In [ ]:
install.packages("rpart")
library(rpart)
rpart.model <- rpart(Wine ~ .,data = train.data, method = "anova")
rpart.model

#transform test into PCA
test.data <- predict(W.pca, newdata = test_set)
test.data <- as.data.frame(test.data)

#select the first 30 components
test.data <- test.data[,1:7]

#make prediction on test data
rpart.prediction <- predict(rpart.model, test.data)

In [ ]:
rpart.prediction

In [ ]:
l=test_set$Wine-rpart.prediction

Decision Tree classifier was used to classify the data after applying PCA.

Now, we classify without applying PCA.

In [ ]:
library(rpart)
rpart.model <- rpart(Wine ~ .,data = training_set, method = "anova")
rpart.model

#make prediction on test data
rpart.prediction <- predict(rpart.model, testing_set)

In [ ]:
rpart.prediction

In [ ]:
k=test_set$Wine-rpart.prediction

In [ ]:
k-l